# Bot Licitaciones

El objetivo es desarrollar un bot que publique de una manera sencilla la información de las licitaciones del Estado. Para eso:

- [X] Entramos a comprar.gob.ar
- [X] Listamos las últimas licitaciones
- [X] Ingresamos a los detalles de cada una
- [X] Si hay cuadro comparativo, ingresamos para buscar el monto
- [ ] Ordenamos los datos
- [ ] Damos formato a la información
- [ ] Publicamos en Twitter

In [3]:
import os
import re
import pandas as pd
from time import sleep
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from IPython.display import HTML, display

In [4]:
def init_chrome():
    """Funcion para iniciar Chrome"""
    
    chrome_options = Options()
    
    useragent = (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like "
        "Gecko) Chrome/73.0.3683.103 Safari/537.36"
    )
    
    # En windows C:\Users\<username>\AppData\Local\Google\Chrome\User Data\Default
    
    datadir = r'/home/eugenio/Global_Shapers/Bot_Comprar/OIA/TwitterBought'
    chrome_options.add_argument(f"user-data-dir={datadir}") if datadir else None
    chrome_options.add_argument(f"user-agent={useragent}") if useragent else None
    chrome_options.add_argument("start-maximized")
    # chrome_options.add_argument("--headless")
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument('disable-blink-features=AutomationControlled')
        
    driver = Chrome(
        executable_path=os.path.join(os.getcwd(), 'chromedriver'), # esto puede ser chromedriver.exe si estas en windows
        options=chrome_options,
        desired_capabilities=chrome_options.to_capabilities(),
    )
    return driver

In [5]:
def extract_data(driver, th=0.5, verbose=True):
    """Para extraer tablas del HTML"""
    #global driver
    dfs = pd.read_html(driver.page_source)
    data = []
    for df in dfs:
        if (df.isnull().sum().sum() / df.size > th):
            del df
            continue

        if verbose: display(HTML(df.to_html()))
        data.append(df.to_dict('list'))
    return data

In [6]:
def extract_panels(driver):
    """Para las tablas en el otro formato"""
    tables = driver.find_elements_by_xpath('//div[contains(@class, "panel")]')

    tablas = []
    for t in tables:
        try:
            title = t.find_element_by_xpath('.//h4').text
        except:
            title=None
        # iteramos tabla
        datos_tabla = []
        for r in t.find_elements_by_xpath('.//div[contains(@class, "row")]'):
            cols = r.find_elements_by_xpath('.//div[contains(@class, "col")]')
            row = {}
            for c in cols:
                try:
                    dato = c.find_element_by_xpath('.//span').text
                except:
                    dato=None
                    continue
                try:
                    label = c.find_element_by_xpath('.//label').text
                except:
                    label = title # mm

                row[label]=dato        

            try:
                del row['']
            except:
                pass
            if len(row) > 0:
                datos_tabla.append(row)

        if len(datos_tabla)>0:
            tablas.append(datos_tabla)
    return tablas

In [7]:
url = 'https://comprar.gob.ar/'
pausa = 3

In [8]:
driver = init_chrome()
driver.get(url)
sleep(pausa)

# xpath cheatsheet https://devhints.io/xpath
driver.find_element_by_xpath('//*[contains(text(), "Ver todos")]').click()

# primera tabla con las 
df = pd.read_html(driver.page_source)[0]
df = df.drop([c for c in df.columns if 'Unnamed' in c], axis=1).iloc[:-2, :]
display(HTML(df.to_html()))
print('--'*50)

,Número de Proceso,Nombre descriptivo de Proceso,Tipo de Proceso,Fecha de Apertura,Estado,Unidad Ejecutora,Servicio Administrativo Financiero
0,46/18-0391-CDI21,SEGUNDA REPARACIÓN DE MOVILIDADES VARIAS,Contratación Directa,26/05/2021 07:00 Hrs.,Publicado,46/18 - 18° Chaco - DNV,604 - Dirección Nacional de Vialidad
1,38/35-0442-LPR21,"Servicio de revisión, recorrido, recarga de matafuegos de la ESNP Y ESBU",Licitación Privada,26/05/2021 08:00 Hrs.,Publicado,38/35 - Escuela Nacional de Pesca - Armada Argentina,379 - Estado Mayor General de la Armada
2,46/22-0380-CDI21,PROVISIÓN E INSTALACIÓN DE TORRE DE ANTENA DE 42 M CON BALIZADO Y PARARRAYOS,Contratación Directa,26/05/2021 08:00 Hrs.,Publicado,46/22 - 22° Formosa - DNV,604 - Dirección Nacional de Vialidad
3,40/22-0130-LPR21,MANTENIMIENTO Y REPARACIÓN DE TECHO,Licitación Privada,26/05/2021 08:00 Hrs.,Publicado,40/022 - UOC DIL,381 - Estado Mayor General de La Fuerza Aérea
4,14/3-0274-CDI21,"INSUMOS ELECTRICIDAD,FERRETERIA Y HERRAMIENTAS PARA TALLERES",Contratación Directa,26/05/2021 08:00 Hrs.,Publicado,14/3 - Centro Atómico Bariloche - CNEA,105 - Comisión Nacional de Energía Atómica
5,84/20-0735-LPR21,ADQUISICIÓN DE SERVICIO DE MANTENIMIENTO Y REPARACIÓN PARA PEUGEOT PARTNER,Licitación Privada,26/05/2021 08:00 Hrs.,Publicado,84/20 - Comando Brigada Mecanizada IX,374 - Estado Mayor General del Ejercito
6,74/26-0149-LPR21,ADQUISICIÓN DE PINTURAS Y LIJAS,Licitación Privada,26/05/2021 08:00 Hrs.,Publicado,74/26 - Parque Nacional Mburucuya,107 - Administración de Parques Nacionales
7,38/22-0111-LPU21,Adquisición de ferretería e insumos de electricidad para el Apostadero San Fernando.,Licitación Pública,26/05/2021 08:00 Hrs.,Publicado,38/22 - Secretaria General de la Armada,379 - Estado Mayor General de la Armada
8,84/34-0080-CDI21,ADQUISICION DE ARTICULOS DE PROTOCOLO Y CEREMONIAL -,Contratación Directa,26/05/2021 08:00 Hrs.,Publicado,84/34 - Regimiento de Granaderos a Caballo,374 - Estado Mayor General del Ejercito
9,84/34-0706-LPR21,ADQUISICION DE ART DE FERRETERIA PARA EL 2DO TRIM 2021,Licitación Privada,26/05/2021 08:00 Hrs.,Publicado,84/34 - Regimiento de Granaderos a Caballo,374 - Estado Mayor General del Ejercito


----------------------------------------------------------------------------------------------------


In [9]:
data = []
# para cada licitacion
for n in range(10):
    data_licitacion = []
    # seleccionamos licitacion
    elem = driver.find_elements_by_xpath('//tr/td[1]/a')[n]
    # inicializamos "base" con el id
    licitacion_id = elem.text
    elem.click()
    sleep(pausa)
    
    # vemos solo las tablas de la primera
    data_licitacion.extend(extract_data(driver=driver, verbose=True if n == 0 else False))    
    data_licitacion.extend(extract_panels(driver=driver))
    
    # dice monto?
    if 'Ofertas al proceso de compra' in driver.page_source:
        driver.find_element_by_xpath("//*[contains(text(), 'cuadro comparativo')]").click()
        data_licitacion.extend(extract_data(verbose=True if n == 0 else False))
        data_licitacion.extend(extract_panels())
        driver.back()
        
    posibles_tablas = [e.text for e in driver.find_elements_by_xpath('//h4') if len(e.text) > 0]
    data.append(data_licitacion)

    driver.back()
    sleep(pausa)
    print('--'*50)
    break

,Número solicitud de contratación,Estado,Unidad Ejecutora,Rubro,Tipo de urgencia,Fecha creación
0,46/18-672-SCO21,Autorizada en Proceso,46/18 - 18° Chaco - DNV,MANT. REPARACION Y LIMPIEZA,Normal,23/04/2021


,Número renglón,Objeto del gasto,Código del ítem,Descripción,Cantidad,Acciones
0,1,3.3.2,3.3.2-5266.21,"MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULOS, CRITERIO SOCIAL: SIN","1,00 SERVICIO",NaN
1,2,3.3.2,3.3.2-5266.21,"MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULOS, CRITERIO SOCIAL: SIN","1,00 SERVICIO",NaN
2,3,3.3.2,3.3.2-5266.21,"MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULOS, CRITERIO SOCIAL: SIN","1,00 SERVICIO",NaN
3,4,3.3.2,3.3.2-5266.21,"MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULOS, CRITERIO SOCIAL: SIN","1,00 SERVICIO",NaN
4,5,3.3.2,3.3.2-5266.21,"MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULOS, CRITERIO SOCIAL: SIN","1,00 SERVICIO",NaN
5,6,3.3.2,3.3.2-5266.21,"MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULOS, CRITERIO SOCIAL: SIN","1,00 SERVICIO",NaN


,Pliego de Bases y Condiciones Generales,Disposición aprobatoria,Fecha creación
0,IF-2019-98574914-APN-DNCBYS#JGM,DI-2016-01714551-APN-ONC#MM,05/11/2019


,Nº de requisito,Descripción,Tipo de documento
0,1,SEGÚN PLIEGO,NaN


,Nº de requisito,Descripción,Tipo de documento
0,1,SEGÚN ESPECIFICACIONES TÉCNICAS,NaN


...

,Documento,Número GDE,Número especial,Fecha vinculación,Opciones
0,Clausulas Particulares,PLIEG-2021-39047828-APN-DCHA#DNV,NaN,04/05/2021,NaN


,Nº penalidad,Descripción
0,1,SEGÚN ART 29° DEL DECRETO 1023/01 Y TITULO V DEL DECRETO 1030/16


,Nombre del Anexo,Tipo,Descripción,Acciones
0,IF-2021-34628592-APN-DCHA%DNV-EETT.pdf,Técnico,ANEXO DE ESPECIFICACIONES TÉCNICAS,NaN


,Documento,Número GDE,Número especial,Fecha vinculación,Opciones
0,Autorización pliego,DI-2021-42725356-APN-DCHA#DNV,DI-2021-84-DCHA#DNV,14/05/2021,NaN
1,Autorización llamado,DI-2021-42725356-APN-DCHA#DNV,DI-2021-84-DCHA#DNV,14/05/2021,NaN


----------------------------------------------------------------------------------------------------


In [10]:
data

[[{'Número solicitud de contratación': ['46/18-672-SCO21'],
   'Estado': ['Autorizada en Proceso'],
   'Unidad Ejecutora': ['46/18 - 18° Chaco - DNV'],
   'Rubro': ['MANT. REPARACION Y LIMPIEZA'],
   'Tipo de urgencia': ['Normal'],
   'Fecha creación': ['23/04/2021']},
  {'Número renglón': [1, 2, 3, 4, 5, 6],
   'Objeto del gasto': ['3.3.2', '3.3.2', '3.3.2', '3.3.2', '3.3.2', '3.3.2'],
   'Código del ítem': ['3.3.2-5266.21',
    '3.3.2-5266.21',
    '3.3.2-5266.21',
    '3.3.2-5266.21',
    '3.3.2-5266.21',
    '3.3.2-5266.21'],
   'Descripción': ['MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULOS, CRITERIO SOCIAL: SIN',
    'MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULOS, CRITERIO SOCIAL: SIN',
    'MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULOS, CRITERIO SOCIAL: SIN',
    'MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULOS, CRITERIO SOCIAL: SIN',
    'MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULO

## Como indexar los resultados del web scrapping:

data es una lista de una lista de diccionarios, con el primer indice [[numero]] navegas sobre los distintos objetos que scrappeamos, es decir: las distintas licitaciones que hizo el gobierno, con eso lo convertis en una lista con algunas listas de diccionarios. data[[numero]]

Con el segundo indice [[numero]] navegas al rededor de los distintos diccionarios dentro de esa lista, que corresponden a distintos elementos de la licitacion. Proba con distintos valores para ver como funciona y que trae. Con esto lo convertis en una lista de diccionarios. data[[numero]][[numero]]

Con el tercer indice [[numero]] navegas dentro de los distintos diccionarios dentro de la lista que devolvio el segundo indice elegido y podes elegir que diccionario quedarte. Algunas listas tienen solo un diccionario por lo cual no se puede utilizar este indice. Proba con distintos valores para ver que trae. data[[numero]][[numero]][[numero]]

Elegi quedarme con informacion del proceso, la moneda y el alcance que tiene y los detalles de todos los productos.

In [11]:
df_info_proceso = pd.DataFrame.from_dict(data=data[0][12][0], orient='index')
df_moneda_y_alcance = pd.DataFrame.from_dict(data[0][12][2], orient='index')
df_detalle_productos = pd.DataFrame(data[0][1])

Armo variables que contengan los valores que quiero y listas sobre los detalles de los productos.

In [12]:
alcance = df_moneda_y_alcance[0][0]
moneda = df_moneda_y_alcance[0][1]
titulo = df_info_proceso[0][1]

In [13]:
productos = []
cantidad = []
for item in range (len(df_detalle_productos)):

    desc = df_detalle_productos.iloc[item]['Descripción']
    productos.append(desc)

    q = df_detalle_productos.iloc[item]['Cantidad']
    q = q.split(' ')[0]
    cantidad.append(q)

Cargo todo a un diccionario para poder manipularlo con mayor facilidad.

Me quedo con los valores unicos en la lista de productos y cantidad para no twittear twitts iguales.

In [30]:
data_processed = {
    'alcance': df_moneda_y_alcance[0][0],
    'moneda': df_moneda_y_alcance[0][1],
    'titulo': df_info_proceso[0][1],
    'productos': list(set(productos)),
    'cantidad': list(set(cantidad))
}

data_processed

{'alcance': 'Nacional',
 'moneda': 'Peso Argentino',
 'titulo': 'SEGUNDA REPARACIÓN DE MOVILIDADES VARIAS',
 'productos': ['MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULOS, CRITERIO SOCIAL: SIN'],
 'cantidad': ['1,00']}

## Tweet
Creo el formato de los tweets con emojis.

In [31]:
import emoji

In [32]:
# Ejemplo de uso:
t = emoji.emojize(f'Python is :thumbs_up: {moneda} :dollar:', use_aliases=True)
print(t)

Python is 👍 Peso Argentino 💵


Ver lista de emojis aca: [Emojis Cheat Sheet](https://www.webfx.com/tools/emoji-cheat-sheet/)

In [33]:
for item in range (len(data_processed['productos'])):

    tweet = emoji.emojize(
        f"""
    \033[1m:package:Titulo:\033[0m {data_processed['titulo']}\n
    \033[1m:dollar:Moneda:\033[0m {data_processed['moneda']}\n
    \033[1m:earth_americas:Alcance:\033[0m {data_processed['alcance']}\n
    \033[1m:hospital:Producto:\033[0m {data_processed['productos'][item]}\n
    \033[1m:white_check_mark:Cantidad:\033[0m {data_processed['cantidad'][item]}
    """, use_aliases=True)
    print(tweet)


    📦Titulo: SEGUNDA REPARACIÓN DE MOVILIDADES VARIAS

    💵Moneda: Peso Argentino

    🌎Alcance: Nacional

    🏥Producto: MANT. Y REP. DE VEHICULOS; DESCRIPCION: MANT. Y REP. DE VEHICULOS, CRITERIO SOCIAL: SIN

    ✅Cantidad: 1,00
    


# Notas

- Ver clasificadas como COVID  
- Objeto  
- Organismos  
- Estandarizar la información en una estructura (obj, monto, a quien...) 
- Humanizarlo (NLG, y personaje): emojis? personalidad, marca, identidad digital
- Check moneda == peso


- https://www.buenosairescompras.gob.ar/